In [ ]:
!pip install numba umap-learn

In [ ]:
import polars as pl
import numpy as np
import pandas as pd
import seaborn as sns
import hdmedians as hdm

import rmsd_map
from rmsd_map.mol_io.cor_reader import read_cor_file
from rmsd_map.mol_io.fragment import Fragment
from rmsd_map.rmsd.pipelines import align_fragments


from bokeh.plotting import figure, show
from bokeh.palettes import Turbo256
from bokeh.models import ColumnDataSource, HoverTool, ColorBar, LinearColorMapper
import plotly.express as px
import numpy as np
from bokeh.io import output_notebook
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

output_notebook()

def embedding_plotter(embedding, data=None, hue=None, hover=None):
    '''
    Рисовалка эмбеддинга. 2D renderer: bokeh. 3D renderer: plotly.
    '''
    if embedding.shape[1] not in [2, 3]:
        raise ValueError("Embedding must be 2D or 3D")

    plot_data = pd.DataFrame()
    plot_data['x'] = embedding[:, 0]
    plot_data['y'] = embedding[:, 1]

    if embedding.shape[1] == 3:
        plot_data['z'] = embedding[:, 2]

    if hover is not None:
        if isinstance(hover, str):
            hover = [hover]
        for col in hover:
            if data is not None:
                plot_data[col] = data[col].values

    if embedding.shape[1] == 2:
        p = figure(width=800, height=600, tools=['pan', 'box_zoom', 'reset'])

        if hue is not None and data is not None:
            # Непрерывная цветовая палитра
            color_mapper = LinearColorMapper(palette=Turbo256,
                                          low=data[hue].min(),
                                          high=data[hue].max())

            source = ColumnDataSource({
                'x': plot_data['x'],
                'y': plot_data['y'],
                'hue': data[hue]
            })

            if hover:
                for col in hover:
                    source.data[col] = data[col].values

            p.scatter('x', 'y',
                     source=source,
                     size=8,
                     color={'field': 'hue', 'transform': color_mapper},
                     alpha=0.6)

            color_bar = ColorBar(color_mapper=color_mapper,
                               label_standoff=12,
                               title=hue)
            p.add_layout(color_bar, 'right')

            if hover:
                tooltips = [(col, '@' + col) for col in hover]
                tooltips.append((hue, '@hue'))
                hover_tool = HoverTool(tooltips=tooltips)
                p.add_tools(hover_tool)

        else:
            p.scatter(plot_data['x'], plot_data['y'], size=8, alpha=0.6)

            if hover:
                tooltips = [(col, '@' + col) for col in hover]
                hover_tool = HoverTool(tooltips=tooltips)
                p.add_tools(hover_tool)

        show(p)

    # 3D
    else:
        if hue is not None and data is not None:
            fig = px.scatter_3d(plot_data, x='x', y='y', z='z',
                              color=data[hue],
                              hover_data=hover if hover else None)
        else:
            fig = px.scatter_3d(plot_data, x='x', y='y', z='z',
                              hover_data=hover if hover else None)

        fig.update_layout(
            width=800,
            height=600,
        )


        fig.show()
def representative_point_idx(df):
        points = df.select(pl.col("X", "Y")).to_numpy()
        median = hdm.geomedian(points, axis = 0)
        dists = np.linalg.norm(points - median, axis=1)
        return np.argmin(dists)
import sklearn.cluster as clu

from IPython.display import display, HTML

To generate UMAP files from `hexanes_rwp5_constr.cor` run

``` 
 rmsd-map-distances -o hexanes_rwp5_constr hexanes_rwp5_constr.cor
 rmsd-map-umaps -o hexanes_rwp5_constr_umaps hexanes_rwp5_constr.npz
 rmsd-map-umaps -d -o hexanes_rwp5_constr_umaps_d hexanes_rwp5_constr.npz
```

In [ ]:
cor = read_cor_file("./acids_all_noh.cor")
cor = np.asarray(cor, dtype=object)
um = pl.read_csv("./acids_all_noh_umaps.csv") # Vanilla UMAP
ud = pl.read_csv("./acids_all_noh_densmap.csv") # Denity-preserving UMAP 



df = ud.filter(pl.col("N") == 30)
sns.scatterplot(data = df , x="X", y="Y")

In [ ]:
dbscan = clu.HDBSCAN(min_cluster_size=30,
    min_samples=15,
    cluster_selection_epsilon=1,
    metric='euclidean').fit(df.select(pl.col("X", "Y")).to_numpy())
df2 = df.with_columns(pl.Series("label", dbscan.labels_))

embedding = df2.select(pl.col("X", "Y")).to_numpy()
embedding_plotter(
    embedding,
    data=df2.to_pandas(),
    hue='label',
    hover = ['label']
)

In [ ]:
for i in df2["label"].unique():
    clu0 = cor[df2["label"] == i]
    clu0_center_idx = representative_point_idx(df2.filter(pl.col("label") == i) ) # find a central point on umap
    clu0_aligned = align_fragments(clu0, clu0_center_idx) # and align all fragments to it
    print(f'Cluster {i} has {len(clu0_aligned)} fragments')
    clu0_view = Fragment.plot_fragments(clu0_aligned)
    display(HTML(clu0_view.write_html(fullpage=True)))


In [ ]:
from umap import UMAP

import warnings
warnings.filterwarnings('ignore')

with np.load('./acids_all_noh.npz') as data:
        names = data['names']
        distances = data["distances"]


reducer = UMAP(metric='precomputed',
                   n_neighbors=45,
                   densmap=True,
                   random_state=42,
                   n_components = 3)
reducer.fit(distances)
df = pl.DataFrame(reducer.embedding_, schema=["X", "Y", 'Z'])
df = df.with_columns(pl.lit(30).alias("N"))


dbscan = clu.HDBSCAN(min_cluster_size=10,
    min_samples=10,
    cluster_selection_epsilon=0.1,
    metric='manhattan').fit(df.select(pl.col("X", "Y", "Z")).to_numpy())
df2 = df.with_columns(pl.Series("label", dbscan.labels_))



In [ ]:

embedding = df2.select(pl.col("X", "Y", "Z")).to_numpy()

embedding_plotter(
    embedding,
    data=df2.to_pandas(),
    hue='label',
    hover = ['label']
)

In [ ]:
for i in df2["label"].unique():
    clu0 = cor[df2["label"] == i]
    clu0_center_idx = representative_point_idx(df2.filter(pl.col("label") == i) ) # find a central point on umap
    clu0_aligned = align_fragments(clu0, clu0_center_idx) # and align all fragments to it
    print(f'Cluster {i} has {len(clu0_aligned)} fragments')
    clu0_view = Fragment.plot_fragments(clu0_aligned)
    display(HTML(clu0_view.write_html(fullpage=True)))
